<a href="https://colab.research.google.com/github/Vac1k/Aurora_Project/blob/main/AuroraModelWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!ngrok config add-authtoken 2tPMHBHBtoCyuWXLK2cgkhcpNvl_3g3KYNbVTLmfjr4QgyvQ7
!pip install flask pyngrok
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api

app = Flask(__name__)

file_path_inventory = "/content/Залишки по ТТ.xlsx"
inventory_df = pd.read_excel(file_path_inventory)
inventory_df = inventory_df.rename(columns={
    "Рівень категорії 3": "Category_Level3",
    "Назва товару": "Product_Full_Name",
    "К-сть залишків на кінець дня": "Stock_Quantity_End_Day",
    "Собівартість залишків на кінець дня": "Stock_Cost_End_Day"
})
inventory_df["Product_ID"] = pd.to_numeric(inventory_df["Product_Full_Name"].str.extract(r'^(\d+)')[0], errors='coerce')
inventory_df["Product_Name"] = inventory_df["Product_Full_Name"].str.replace(r'^\d+\s+', '', regex=True)
inventory_df["Calculated_Price"] = (inventory_df["Stock_Cost_End_Day"] / inventory_df["Stock_Quantity_End_Day"]).round(3)

ai_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
inventory_df["Product_Vector"] = inventory_df["Product_Name"].apply(lambda x: ai_model.encode(x))
word2vec_model = api.load("word2vec-ruscorpora-300")

def find_similar_products(query, top_n=5):
    query_vector = np.array(ai_model.encode(query)).reshape(1, -1)
    product_vectors = np.array(list(inventory_df["Product_Vector"]))
    similarity_scores = cosine_similarity(query_vector, product_vectors).flatten()
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    return inventory_df.iloc[top_indices][["Product_Name", "Category_Level3", "Stock_Quantity_End_Day"]].to_dict(orient="records")

html_template = '''
<!DOCTYPE html>
<html lang="uk">
<head>
    <meta charset="UTF-8">
    <title>AI Рекомендації</title>
    <script>
        async function getRecommendations() {
            const userQuery = document.getElementById("query").value;
            const response = await fetch('/recommend', {
                method: 'POST',
                headers: {'Content-Type': 'application/json'},
                body: JSON.stringify({query: userQuery})
            });
            const data = await response.json();
            document.getElementById("results").innerHTML = data.map((item, i) =>
                `<p><strong>${i+1}.</strong> ${item.Product_Name} - ${item.Category_Level3} - Кількість: ${item.Stock_Quantity_End_Day}</p>`
            ).join('');
        }
    </script>
</head>
<body>
    <h2>🔍 Введіть ваш запит для отримання рекомендацій</h2>
    <input type="text" id="query" placeholder="Введіть запит...">
    <button onclick="getRecommendations()">Пошук</button>
    <div id="results"></div>
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(html_template)

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.json
    query = data.get('query', '')
    recommendations = find_similar_products(query, top_n=5)
    return jsonify(recommendations)

port = 5000
public_url = ngrok.connect(port)
print(f"🔗 Посилання для тестування: {public_url}")

app.run(port=port)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🔗 Посилання для тестування: NgrokTunnel: "https://20b0-34-125-119-41.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:09:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:09:22] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:09:38] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:09:48] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:13:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:13:28] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:13:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:13:30] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:13:39] "POST /recommend HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Feb/2025 19:13:58] "POST /recommend 